In [1]:
import pandas as pd
import mpu
from shapely.ops import nearest_points

# Import Tract Centroids geo info

In [147]:
import geopandas as gpd
centroids = gpd.read_file('nhgis0001_shapefile_cenpop2020_us_tract_cenpop_2020/US_tract_cenpop_2020.shp')

In [148]:
centroids.dtypes

GISJOIN         object
GEOID           object
STATEFP         object
COUNTYFP        object
TRACTCE         object
POPULATION       int64
LATITUDE       float64
LONGITUDE      float64
geometry      geometry
dtype: object

In [149]:
centroids.GEOID = centroids.GEOID.astype(int)
# centroids.LATITUDE = centroids.LATITUDE.astype(str)
# centroids.LONGITUDE = centroids.LONGITUDE.astype(str)

### Get a dataframe of all census centroids in nyc

In [150]:
all_new_york = centroids[centroids.STATEFP=="36"]
all_new_york

nyc_ids = all_new_york[(all_new_york['COUNTYFP']=="005") | (all_new_york['COUNTYFP']=="061")| (all_new_york['COUNTYFP']=="047")| (all_new_york['COUNTYFP']=="081")|(all_new_york['COUNTYFP']=="085")]
nyc_ids

,GISJOIN,GEOID,STATEFP,COUNTYFP,TRACTCE,POPULATION,LATITUDE,LONGITUDE,geometry
49833,G3600050000100,36005000100,36,005,000100,3772,40.791709,-73.884503,POINT (1834172.306 583266.196)
49834,G3600050000200,36005000200,36,005,000200,4779,40.811740,-73.859760,POINT (1835669.238 585926.700)
49835,G3600050000400,36005000400,36,005,000400,6272,40.809831,-73.853290,POINT (1836245.299 585843.682)
49836,G3600050001600,36005001600,36,005,001600,5795,40.819555,-73.859196,POINT (1835513.085 586789.151)
49837,G3600050001901,36005001901,36,005,001901,2292,40.806572,-73.927684,POINT (1830272.601 584053.423)
...,...,...,...,...,...,...,...,...,...
54029,G3600850030302,36085030302,36,085,030302,6669,40.623555,-74.161381,POINT (1815907.262 559617.218)
54030,G3600850031901,36085031901,36,085,031901,3262,40.631387,-74.163047,POINT (1815571.377 560439.554)
54031,G3600850031902,36085031902,36,085,031902,5500,40.637768,-74.166825,POINT (1815100.055 561063.307)
54032,G3600850032300,36085032300,36,085,032300,1233,40.634014,-74.173096,POINT (1814683.641 560534.125)


# Using geopandas to find nearst bench for each census tract

### Import bench data

In [152]:
bench = pd.read_csv("data/cleaned_df.csv")
bench.head(2)

,the_geom,ObjectID,BoroCode,BoroName,BoroCD,CounDist,AssemDist,StSenDist,CongDist,SiteID,Category,Installati,Address,GeocodeAdd,Street,CrossStree,Borough,ComDist,BusRoute,BID,Latitude,Longitude,NTAName,FEMAFldz,FEMAFldT,HrcEvac,SeatingTyp,date
0,POINT (-73.98384524707862 40.66433221370896),444,3.0,Brooklyn,306.0,39.0,44.0,21.0,9.0,1921,General,05/09/2013,412 7th Avenue,412 7th Avenue,7th Avenue,13th & 14th Street,Brooklyn,306,B67,Not Applicable,40.664332,-73.983845,Park Slope,X,AREA OF MINIMAL FLOOD HAZARD,NaN,BACKLESS,2013-05-09
1,POINT (-73.98150878289019 40.67476959289031),554,3.0,Brooklyn,306.0,39.0,52.0,25.0,7.0,1003,General,01/17/2013,249 Fifth Avenue,249 Fifth Avenue,5th Avenue,Garfield Street and Carroll Street,Brooklyn,306,B63,Fifth Avenue,40.674770,-73.981509,Park Slope,X,AREA OF MINIMAL FLOOD HAZARD,NaN,BACKLESS,2013-01-17


In [153]:
bench = bench[['the_geom','ObjectID','BoroName','Latitude','Longitude','NTAName','SeatingTyp','date']]
bench.head(2)

,the_geom,ObjectID,BoroName,Latitude,Longitude,NTAName,SeatingTyp,date
0,POINT (-73.98384524707862 40.66433221370896),444,Brooklyn,40.664332,-73.983845,Park Slope,BACKLESS,2013-05-09
1,POINT (-73.98150878289019 40.67476959289031),554,Brooklyn,40.674770,-73.981509,Park Slope,BACKLESS,2013-01-17


In [154]:
# Get rid of benches that don't have a coordinate
all_bench = bench[bench.Latitude.notna()]
all_bench.shape

(2239, 8)

### remove unwanted bench 

In [155]:
# remove leaning bars
all_bench = all_bench[all_bench.SeatingTyp!="LEANING BAR"]


accessible_bench = all_bench[all_bench.SeatingTyp!="Unknown"]
accessible_bench.shape

(2151, 8)

In [156]:
# take two df arrays arrays and return an array of tuples
accessible_bench['lat_long'] = accessible_bench[['Latitude', 'Longitude']].apply(tuple, axis=1)
all_benches_list_of_tuples = accessible_bench['lat_long'].tolist()
all_benches_list_of_tuples

/var/folders/0h/2v5mxpvs49s0_gs3lqjt474h0000gn/T/ipykernel_40915/2326494127.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accessible_bench['lat_long'] = accessible_bench[['Latitude', 'Longitude']].apply(tuple, axis=1)


[(40.6643322137, -73.9838452471),
 (40.6747695929, -73.9815087829),
 (40.7255692666, -73.9776555731),
 (40.8622731579, -73.8971776619),
 (40.6844349835, -73.8460384939),
 (40.7134457797, -73.8307321104),
 (40.5966695132, -73.9766727149),
 (40.7745277298, -73.9557485634),
 (40.832981806, -73.9453608742),
 (40.7389380192, -73.7013737252),
 (40.8211657158, -73.8185016047),
 (40.8065930596, -73.9385104781),
 (40.7043048415, -73.9864704101),
 (40.8958567042, -73.892292577),
 (40.7765360462, -73.7836181709),
 (40.8459534725, -73.9363351757),
 (40.7732159451, -73.9068629716),
 (40.7895488382, -73.9791923499),
 (40.683040748, -73.9678225652),
 (40.7364350479, -73.8782616219),
 (40.7978104765, -73.9419196642),
 (40.652780664, -73.9525934674),
 (40.7804751026, -73.9767521794),
 (40.6107813709, -74.0086711035),
 (40.8688025063, -73.8968418793),
 (40.7550288101, -73.7403841406),
 (40.7514997294, -73.8260906073),
 (40.6984737728, -73.7522488985),
 (40.602988756, -73.753181639),
 (40.610164726, -73.

### Set up a function to find nearest bench to each centroid

In [157]:
# Calculating closest to a point
# Pass a list of locations as data, and the point that's reference
# data is expected a list of tuples
# point is lat, lon tuple as a float
def get_closest(data, origin):
    return min(data, key=lambda x: find_distance(origin, x))

# Calculating distance from one point to another
# origin and destination points is lat, lon tuple as a float
def find_distance(origin, destination):
    return mpu.haversine_distance(origin, destination)

In [158]:
# Feed tract data into function that finds the nearest point
# With list of bench coordinates as reference
# And create a new column with the nearest bench to each bench's centroids
tract_data["closest_bench"] = tract_data.apply\
    (lambda row: get_closest(all_benches_list_of_tuples, (row["LATITUDE"], row["LONGITUDE"])), axis=1)

### Feed nyc census ids to the function

In [159]:
nyc_ids["closest_bench"] = nyc_ids.apply\
    (lambda row: get_closest(all_benches_list_of_tuples, (row["LATITUDE"], row["LONGITUDE"])), axis=1)

/Users/tuxinyi/.pyenv/versions/3.11.0rc2/lib/python3.11/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [160]:
nyc_ids

,GISJOIN,GEOID,STATEFP,COUNTYFP,TRACTCE,POPULATION,LATITUDE,LONGITUDE,geometry,closest_bench
49833,G3600050000100,36005000100,36,005,000100,3772,40.791709,-73.884503,POINT (1834172.306 583266.196),"(40.8098290466, -73.8802074781)"
49834,G3600050000200,36005000200,36,005,000200,4779,40.811740,-73.859760,POINT (1835669.238 585926.700),"(40.8135012283, -73.8580258069)"
49835,G3600050000400,36005000400,36,005,000400,6272,40.809831,-73.853290,POINT (1836245.299 585843.682),"(40.8135012283, -73.8580258069)"
49836,G3600050001600,36005001600,36,005,001600,5795,40.819555,-73.859196,POINT (1835513.085 586789.151),"(40.8204551606, -73.8580429709)"
49837,G3600050001901,36005001901,36,005,001901,2292,40.806572,-73.927684,POINT (1830272.601 584053.423),"(40.8060822771, -73.9259454483)"
...,...,...,...,...,...,...,...,...,...,...
54029,G3600850030302,36085030302,36,085,030302,6669,40.623555,-74.161381,POINT (1815907.262 559617.218),"(40.6266900646, -74.1619032256)"
54030,G3600850031901,36085031901,36,085,031901,3262,40.631387,-74.163047,POINT (1815571.377 560439.554),"(40.626958394, -74.164303149)"
54031,G3600850031902,36085031902,36,085,031902,5500,40.637768,-74.166825,POINT (1815100.055 561063.307),"(40.6274662085, -74.1677678754)"
54032,G3600850032300,36085032300,36,085,032300,1233,40.634014,-74.173096,POINT (1814683.641 560534.125),"(40.6274662085, -74.1677678754)"


### Merging with benches to get a dataframe of each census tract, nearest bench, and detail of that bench

In [161]:
data =pd.merge\
(nyc_ids, accessible_bench, how="left", left_on="closest_bench", right_on="lat_long")

In [164]:
data.head(2)

,GISJOIN,GEOID,STATEFP,COUNTYFP,TRACTCE,POPULATION,LATITUDE,LONGITUDE,geometry,closest_bench,the_geom,ObjectID,BoroName,Latitude,Longitude,NTAName,SeatingTyp,date,lat_long
0,G3600050000100,36005000100,36,005,000100,3772,40.791709,-73.884503,POINT (1834172.306 583266.196),"(40.8098290466, -73.8802074781)",POINT (-73.88020747806898 40.80982904656402),1512,Bronx,40.809829,-73.880207,Hunts Point,BACKED,2017-06-29,"(40.8098290466, -73.8802074781)"
1,G3600050000200,36005000200,36,005,000200,4779,40.811740,-73.859760,POINT (1835669.238 585926.700),"(40.8135012283, -73.8580258069)",POINT (-73.85802580692271 40.81350122828774),1531,Bronx,40.813501,-73.858026,Soundview-Clason Point,BACKLESS,2014-12-18,"(40.8135012283, -73.8580258069)"


# Using GOOGLE MAPS API to calculate travel time from each tract to nearest bench

In [165]:
import requests
import time

### Build a function to store googlemaps requests into dataframe

In [ ]:
import config
def get_travel_data(row): 
    ori_lat = row['LATITUDE']
    ori_long = row['LONGITUDE']
    des_lat = row['Latitude']
    des_long = row['Longitude']
    
    url = f"https://maps.googleapis.com/maps/api/distancematrix/json?origins={ori_lat}%2C{ori_long}&destinations={des_lat}%2C{des_long}&mode=walking&key={config.api_key}"

    payload={}
    headers = {}
#     Pause between requests
    time.sleep(1)
    
    response = requests.request("GET", url, headers=headers, data=payload)
    result_dic = response.json()
#     store values to dataframe
    try:
        row['distance_km'] = result_dic['rows'][0]['elements'][0]['distance']['text']
        row['distance_m'] = result_dic['rows'][0]['elements'][0]['distance']['value']
        row['travel_time'] = result_dic['rows'][0]['elements'][0]['duration']['text']
        row['travel_time_sec'] = result_dic['rows'][0]['elements'][0]['duration']['value']
        print("SUCCESS")
    except:
        print("NOT FOUND")
        row['distance_km'] = "NA"
        row['distance_m'] = "NA"
        row['travel_time'] = "NA"
        row['travel_time_sec'] = "NA"
        

    
    return row

### Looping thru all rows in tract data

In [ ]:
# # Uncomment this cell to request API
# nyc_all_tracts = data.apply(get_travel_data, axis=1)

In [167]:
nyc_all_tracts.head(2)

,GISJOIN,GEOID,STATEFP,COUNTYFP,TRACTCE,POPULATION,LATITUDE,LONGITUDE,geometry,closest_bench,the_geom,ObjectID,BoroName,Latitude,Longitude,NTAName,SeatingTyp,date,lat_long,distance_km,distance_m,travel_time,travel_time_sec
0,G3600050000100,36005000100,36,005,000100,3772,40.791709,-73.884503,POINT (1834172.306 583266.196),"(40.8098290466, -73.8802074781)",POINT (-73.88020747806898 40.80982904656402),1512,Bronx,40.809829,-73.880207,Hunts Point,BACKED,2017-06-29,"(40.8098290466, -73.8802074781)",NA,NA,NA,0
1,G3600050000200,36005000200,36,005,000200,4779,40.811740,-73.859760,POINT (1835669.238 585926.700),"(40.8135012283, -73.8580258069)",POINT (-73.85802580692271 40.81350122828774),1531,Bronx,40.813501,-73.858026,Soundview-Clason Point,BACKLESS,2014-12-18,"(40.8135012283, -73.8580258069)",0.4 km,376,5 mins,NaN


In [166]:
nyc_all_tracts.to_csv("data/tracts_to_nearest_bench.csv", index=False)